<a href="https://colab.research.google.com/github/vovaekb/ml_playground/blob/master/src/cifar_classification_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras_sequential_ascii

In [0]:
'''
Train a simple deep CNN on the CIFAR10 small images dataset
'''

from __future__ import print_function
from matplotlib import pyplot as plt
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import SGD
from keras.constraints import maxnorm
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.utils import np_utils
from keras_sequential_ascii import sequential_model_to_ascii_printout
from keras import backend as K
import os
import numpy as np
import time


Example: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py

Tutorial: https://blog.plon.io/tutorials/cifar-10-classification-using-keras-tutorial/

In [0]:
# Declare variables
batch_size = 32

num_classes = 10
epochs = 10

data_augmentation = True

In [0]:
# Define image dimensionalty ordering
# For TensorFlow 0.10 would use "tf", 0.11 would use "th"
if K.backend() == 'tensorflow':
  K.set_image_dim_ordering('tf')

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [0]:
print(y_train[0,])

[6]


In [0]:
# Print 10 random images from each class
'''
fig = plt.figure(figsize=(8, 3))
for i in range(num_classes):
  ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
  idx = np.where(x_train[:]==i)[0]
  features_idx = x_train[idx,::]
  img_num = np.random.randint(features_idx.shape[0])
  im = np.transpose(features_idx[img_num,::],(1,2,0))
  ax.set_titles(class_names[i])
  plt.imshow(im)
plt.show()
'''

In [0]:
# Convert and pre-processing
# Question: why sometimes one uses numpy reshape() method to reshape the data samples to (num_samples, 1024)?
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
print(x_train[0,:40])
print(y_train[0])

[[[0.23137255 0.24313726 0.24705882]
  [0.16862746 0.18039216 0.1764706 ]
  [0.19607843 0.1882353  0.16862746]
  ...
  [0.61960787 0.5176471  0.42352942]
  [0.59607846 0.49019608 0.4       ]
  [0.5803922  0.4862745  0.40392157]]

 [[0.0627451  0.07843138 0.07843138]
  [0.         0.         0.        ]
  [0.07058824 0.03137255 0.        ]
  ...
  [0.48235294 0.34509805 0.21568628]
  [0.46666667 0.3254902  0.19607843]
  [0.47843137 0.34117648 0.22352941]]

 [[0.09803922 0.09411765 0.08235294]
  [0.0627451  0.02745098 0.        ]
  [0.19215687 0.10588235 0.03137255]
  ...
  [0.4627451  0.32941177 0.19607843]
  [0.47058824 0.32941177 0.19607843]
  [0.42745098 0.28627452 0.16470589]]

 ...

 [[0.8156863  0.6666667  0.3764706 ]
  [0.7882353  0.6        0.13333334]
  [0.7764706  0.6313726  0.10196079]
  ...
  [0.627451   0.52156866 0.27450982]
  [0.21960784 0.12156863 0.02745098]
  [0.20784314 0.13333334 0.07843138]]

 [[0.7058824  0.54509807 0.3764706 ]
  [0.6784314  0.48235294 0.16470589]


In [0]:
# TODO: Add data augmentation: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
# Interesting example: https://medium.com/@ksusorokina/image-classification-with-convolutional-neural-networks-496815db12a8

In [0]:
# Define network architecture
# 4-layer network
def four_layer_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  
  sgd = SGD(lr=0.1) #, decay=1e-6, momentum=0.9, nesterov=True)
  
  # Train model
  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

# Source: https://colab.research.google.com/drive/1Q7CGlx0lQa8YxwOsLzsPdMMITAjHHENE#scrollTo=XrisZPa4D1hk
def one_layer_model():
  model = Sequential()
  model.add(Conv2D(32, (3,3), input_shape=x_train.shape[1:]))
  model.add(Conv2D(64, (3,3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
  return model

cnn_n = four_layer_model()
#cnn_n = one_layer_model()
cnn_n.summary()

start = time.time()
if not data_augmentation:
  print('Not using data augmentation.')
  cnn = cnn_n.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)
else:
  print('Using real-time data augmentation.')
  # This will do preprocessing and realtime data augmentation:
  datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.0
  )
  
  # Compute quantities required for feature-wise normalization
  # (std, mean, and principal components if ZCA whitening is applied).
  datagen.fit(x_train)
  
  # Fit the model on the batches generated by datagen.flow().
  cnn = cnn_n.fit_generator(datagen.flow(x_train, y_train,
                                        batch_size=batch_size),
                           epochs=epochs, 
                           validation_data=(x_test, y_test),
                           workers=2)

end = time.time()  
print(end - start)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_31 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_32 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 15, 15, 64)        18496     
__________

ValueError: ignored

## Data augmentation
**Without Data augmentation**
loss: 0.9093 - acc: 0.6823 - val_loss: 0.9387 - val_acc: 0.6720
Accuracy: 67.20%, loss: 0.94

**With Data augmentation**
loss: 1.5397 - acc: 0.4432 - val_loss: 1.2537 - val_acc: 0.5486
Accuracy: 54.86%, loss: 1.25


In [0]:
# Visualizing model structure
sequential_model_to_ascii_printout(cnn_n)

In [0]:
# Plot training and testing process
plt.figure(0)
plt.plot(cnn.history['acc'], 'r')
plt.plot(cnn.history['val_acc'], 'g')
plt.xticks(np.arange(0, 101, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num. of Epochs")
plt.ylabel("Accuracy")
plt.title("Training Accuracy vs Validation Accuracy")
plt.legend(['train', 'validation'])

plt.figure(1)
plt.plot(cnn.history['loss'], 'r')
plt.plot(cnn.history['val_loss'], 'g')
plt.xticks(np.arange(0, 101, 2.0))
plt.rcParams['figure.figsize'] = (8, 6)
plt.xlabel("Num. of Epochs")
plt.ylabel("Loss")
plt.title("Training Loss vs Validation Loss")
plt.legend(['train', 'validation'])

plt.show()

In [0]:
# Final testing accuracy
(loss, acc) = cnn_n.evaluate(x_test, y_test)
print('Accuracy: %.2f%%, loss: %.2f' % (acc*100, loss))

10000/10000 [==============================] - 2s 156us/step
Accuracy: 54.86%, loss: 1.25


In [0]:
# Save model
model_name = "simple_cnn"
cnn_n.save(model_name)
print("Model %s has been successfully saved" % model_name)

Model simple_cnn has been successfully saved
